In [ ]:
import json
import re
import numpy as np
import pandas as pd

ADMIN_ID = 'user5790141925'

In [32]:

# Opening JSON file
liza_f = open('layza.json')

data = json.load(liza_f)
data['messages'][:10]

[{'id': 8231,
  'type': 'message',
  'date': '2023-08-02T10:58:22',
  'date_unixtime': '1690943123',
  'from': '𝐿𝒶𝓎𝓏𝒶❤️\u200d🔥',
  'from_id': 'user273013296',
  'text': 'Как установить PyCharm?',
  'text_entities': [{'type': 'plain', 'text': 'Как установить PyCharm?'}]},
 {'id': 8232,
  'type': 'message',
  'date': '2023-08-02T11:36:22',
  'date_unixtime': '1690965382',
  'from': 'Администратор INNOPROG',
  'from_id': 'user5790141925',
  'photo': '(File not included. Change data exporting settings to download.)',
  'width': 733,
  'height': 1280,
  'text': ['Добрый день! Отвечаем на вопрос: «',
   {'type': 'italic', 'text': 'как скачать файд в pyCharm'},
   '»\nВсе файлы проекта в PyCharm находятся в определённой папке этого проекта. Чтобы понять, где искать файл на устройстве, достаточно нажать правой кнопкой мыши на название открытого файла или сам файл, затем выбрать Open In и вам будет предложено открыть файл в папке. Так откроется папка с проектом, где вы и найдёте открытый файл.'

In [34]:
messages = []
users = set()
for msg in data['messages']:
    users.add(msg['from_id'])
    temp_dict = {
        'id': msg['id'],
        'from': msg['from_id'],
        'text' : "",
        'date' : msg['date_unixtime']
    }
    for t in msg['text_entities']:
        temp_dict['text'] += t['text'] + " "

    temp_dict['text'] = temp_dict['text'].strip()
    messages.append(temp_dict)

messages[:10]

[{'id': 8231,
  'from': 'user273013296',
  'text': 'Как установить PyCharm?',
  'date': '1690943123'},
 {'id': 8232,
  'from': 'user5790141925',
  'text': 'Добрый день! Отвечаем на вопрос: « как скачать файд в pyCharm »\nВсе файлы проекта в PyCharm находятся в определённой папке этого проекта. Чтобы понять, где искать файл на устройстве, достаточно нажать правой кнопкой мыши на название открытого файла или сам файл, затем выбрать Open In и вам будет предложено открыть файл в папке. Так откроется папка с проектом, где вы и найдёте открытый файл.',
  'date': '1690965382'},
 {'id': 8233,
  'from': 'user5790141925',
  'text': 'Скажите, пожалуйста, у Вас получилось? Или нужна еще помощь',
  'date': '1690972578'},
 {'id': 8234,
  'from': 'user273013296',
  'text': 'Получилось, но отправляя файл в телеграм, пишет неверный ответ',
  'date': '1690973284'},
 {'id': 8235,
  'from': 'user5790141925',
  'text': 'Пришлите код, пожалуйста, который написали в файле',
  'date': '1690973344'},
 {'id': 8

In [35]:
len(messages)

443

Grouping multiple messages into one text

In [36]:
grouped = []
text_temp = ''
for idx in range(len(messages)):

    if idx != len(messages)-1 and messages[idx]['from'] == messages[idx + 1]['from']:
        text_temp += messages[idx]['text'] + " "
    else:
        text_temp += messages[idx]['text']
        grouped.append(
                {
                'id': messages[idx]['id'],
                'from': messages[idx]['from'],
                'text': re.sub(r'\s+', ' ', text_temp).strip(),
                'date': messages[idx]['date']
                }
        )
        text_temp = ''

grouped[:5]

[{'id': 8231,
  'from': 'user273013296',
  'text': 'Как установить PyCharm?',
  'date': '1690943123'},
 {'id': 8233,
  'from': 'user5790141925',
  'text': 'Добрый день! Отвечаем на вопрос: « как скачать файд в pyCharm » Все файлы проекта в PyCharm находятся в определённой папке этого проекта. Чтобы понять, где искать файл на устройстве, достаточно нажать правой кнопкой мыши на название открытого файла или сам файл, затем выбрать Open In и вам будет предложено открыть файл в папке. Так откроется папка с проектом, где вы и найдёте открытый файл. Скажите, пожалуйста, у Вас получилось? Или нужна еще помощь',
  'date': '1690972578'},
 {'id': 8234,
  'from': 'user273013296',
  'text': 'Получилось, но отправляя файл в телеграм, пишет неверный ответ',
  'date': '1690973284'},
 {'id': 8235,
  'from': 'user5790141925',
  'text': 'Пришлите код, пожалуйста, который написали в файле',
  'date': '1690973344'},
 {'id': 8237,
  'from': 'user273013296',
  'text': 'Print(“Hello World”)',
  'date': '1690

In [37]:
len(grouped)

214

In [38]:
# Convert to DataFrame
df = pd.DataFrame(grouped)

questions = df[df['from'] == 'user273013296']['text'].tolist()
answers = df[df['from'] == 'user5790141925']['text'].tolist()

answers += [''] * (len(questions) - len(answers))

# Create the final DataFrame
qa_df = pd.DataFrame({
    'question': questions,
    'answer': answers
})

qa_df

,question,answer
0,Как установить PyCharm?,Добрый день! Отвечаем на вопрос: « как скачать...
1,"Получилось, но отправляя файл в телеграм, пише...","Пришлите код, пожалуйста, который написали в ф..."
2,Print(“Hello World”),Отлично! Программа сохранена. Теперь этот файл...
3,добрый день! зачем ставить знак # перед числам...,Добрый день! Это называется комментарием. Этот...
4,"Аааа поняла) Но зачем тогда она вообще нужна, ...",Так как в реальности обычно несколько человек ...
...,...,...
102,ну а как тогда я на другую строку спущусь и на...,"Необязательно спускаться, нужно писать рядом и..."
103,"что писать рядом? написано в задаче: "" пользов...",Ранее Вам никогда не приходилось выводить числ...
104,🤦🏼 Вообще про это на уроках не было Даже в ине...,Это не является каким-либо отдельным приемом. ...
105,"да, это помогло) спасибо) на самом деле я чуто...",Добрый вечер! В данной задаче потребуется все ...


In [ ]:
df.to_csv('datasets/chat/prepared/liza.csv')